In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader
%pip install nltk

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
  Using cached plotly-5.18.0-py3-none-any.whl.metadata (7.0 kB)
Using cached plotly-5.18.0-py3-none-any.whl (15.6 MB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 13.80 GB
Used RAM: 1.34 GB
Percentage Usage Of RAM: 10.8%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 15.22 GB
Used Disk: 9.77 GB
Percentage Usage Of Disk: 39.1%


In [3]:
# import requirements
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from gbm_classifier import Classification

[nltk_data] Downloading package vader_lexicon to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# get the data
boxing = pd.read_csv("boxing.csv")

In [5]:
# prepare the data for machine learning
boxing = boxing.sample(frac=1, random_state=0).reset_index(drop=True)  # shuffle the data
boxing = boxing.ffill().bfill()  # fill in missing values with the last known value 
boxing["age_A"] = boxing["age_A"] + 0.01  # add a small number to age_A so it isn't treated as a categorical variable

In [6]:
# get the testing data
y = boxing[["result"]]
X = boxing.drop(columns="result")
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [7]:
# build the model
print("\n---- Boxing Classification Analysis ----\n")
model = Classification(
    name="XGBoost Without Feature Engineering", 
    path=None,
    rename=False, 
    time=False, 
    text=False,
    binary=True, 
    imputation=False, 
    variance=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    tune=True,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.explore(boxing)
    model.validate(X, y)  # build the machine learning pipeline
    predictions = model.predict(testX)
    print("\nModel Performance:")
    print(f"Accuracy: {model.accuracy}")
    print(f"F1: {model.f1}")
    print(f"In Control: {model.in_control}")
    print("Confusion Matrix:")
    pd.set_option("display.width", 1000)
    print(model.confusion / model.confusion.sum().sum())


---- Boxing Classification Analysis ----

Visualizing The Data:
> Plotting Correlations
> won_A vs. kos_A
> won_B vs. kos_B
> judge1_A vs. judge1_B
> judge1_A vs. judge2_A
> judge1_A vs. judge2_B
> judge1_A vs. judge3_A
> judge1_A vs. judge3_B
> judge1_B vs. judge2_A
> judge1_B vs. judge2_B
> judge1_B vs. judge3_A
> judge1_B vs. judge3_B
> judge2_A vs. judge2_B
> judge2_A vs. judge3_A
> judge2_A vs. judge3_B
> judge2_B vs. judge3_A
> judge2_B vs. judge3_B
> judge3_A vs. judge3_B
> Plotting age_A
> Plotting age_B
> Plotting height_A
> Plotting height_B
> Plotting reach_A
> Plotting reach_B
> Plotting weight_A
> Plotting weight_B
> Plotting won_A
> Plotting won_B
> Plotting lost_A
> Plotting lost_B
> Plotting drawn_A
> Plotting drawn_B
> Plotting kos_A
> Plotting kos_B
> Plotting judge1_A
> Plotting judge1_B
> Plotting judge2_A
> Plotting judge2_B
> Plotting judge3_A
> Plotting judge3_B
> Plotting stance_A
> Plotting stance_B
> Plotting result
> Plotting decision
> stance_A vs. stance_B

In [8]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10].tolist()):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10].tolist()):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Indicators:
1. decision_TD
2. decision_PTS
3. decision_NWS
4. decision_MD
5. decision_SD
6. decision_UD
7. decision_KO
8. decision_TKO
9. decision_DQ
10. decision_RTD
 
Feature Drift:
1. reach_A
2. judge2_A
3. judge3_B
4. judge3_A
5. judge1_A
6. reach_B
7. judge1_B
8. result


In [9]:
# score the model
accuracy = accuracy_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)
f1 = f1_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    average="macro",
)

print(f"Accuracy: {accuracy}")
print(f"F1: {f1}")

Accuracy: 0.8529521842937343
F1: 0.49557208144375015


In [10]:
# show the confusion matrix
ytest = testy.iloc[:,0].to_numpy()
labels = np.unique(np.concatenate((predictions, ytest)))
confusion = confusion_matrix(
    y_true=ytest,   # rows
    y_pred=predictions,  # columns
    labels=labels,
)
confusion = pd.DataFrame(
    confusion, 
    columns=labels, 
    index=labels,
)
print("Confusion Matrix:")
pd.set_option("display.width", 1000)
print(confusion / confusion.sum().sum())

Confusion Matrix:
           draw     win_A     win_B
draw   0.009292  0.053236  0.001833
win_A  0.002400  0.821424  0.005911
win_B  0.001407  0.082261  0.022237


In [11]:
# save the machine learning pipeline
model.dump()

In [12]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The Updated Data
> Transforming Categorical Features
> Removing Constant Features
> Tuning XGBoost
> Cross Validating 15 Models


KeyboardInterrupt: 